In [1]:
pip install facenet-pytorch
pip install -v -e
pip install -U openmim
pip install mmcv
pip install scikit-image
pip install keras

SyntaxError: invalid syntax (479725619.py, line 1)

In [26]:
# importing libraries


import torch
from torchvision import datasets
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.io
import io
from facenet_pytorch import MTCNN, InceptionResnetV1
# from scipy.misc import impip save
from imageio import imwrite
# from google.colab.patches import cv2_imshow

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image, ImageEnhance



In [28]:
# initializing MTCNN and InceptionResnetV1 
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
# mtcnn = MTCNN(keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval() 


In [29]:
import os

data_dir = 'C:/Users/mayan/OneDrive/Desktop/CyberHack/train/train_images' 
os.listdir(data_dir)  
from keras.preprocessing.image import ImageDataGenerator
   
datagen = ImageDataGenerator(
        rotation_range = 40,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        brightness_range = (0.5, 1.5),
        featurewise_center=True,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=True,
        width_shift_range=0.2,
        height_shift_range=0.1,
        channel_shift_range=0.1,
        fill_mode='nearest',
        cval=0.,
        rescale=None,
        preprocessing_function=None)

for image_class in os.listdir(data_dir): 

        i = 0

        dir = "C:/Users/mayan/OneDrive/Desktop/CyberHack/train/train_images/"+image_class
        aug = "C:/Users/mayan/OneDrive/Desktop/CyberHack/train/train_images_augmented/"+image_class

        for batch in datagen.flow_from_directory(directory=dir, batch_size = 8,
                          save_to_dir =aug, 
                          save_prefix ='aug', save_format ='jpg'):
                i += 1
                
                if i > 100:
                        break

Found 17 images belonging to 1 classes.


C:\Users\mayan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\preprocessing\image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
C:\Users\mayan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\preprocessing\image.py:1884: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Found 1 images belonging to 1 classes.


In [30]:
# Read data from folder

dataset = datasets.ImageFolder('C:/Users/mayan/OneDrive/Desktop/CyberHack/train/train_images_augmented') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

# dists = [[(e1 - e2).norm().item() for e2 in embedding_list] for e1 in embedding_list]
# print(pd.DataFrame(dists, columns=name_list, index=name_list))

# save data
data = [embedding_list, name_list] 
torch.save(data, 'C:/Users/mayan/OneDrive/Desktop/CyberHack/data.pt') # saving data.pt file

In [34]:
from skimage.util import img_as_bool
# from google.colab.patches import cv2_imshow
from skimage import exposure
import torch
from torchvision import datasets
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.io
import io
from facenet_pytorch import MTCNN, InceptionResnetV1
# from scipy.misc import impip save
from imageio import imwrite
# from google.colab.patches import cv2_imshow

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image, ImageEnhance

# Using webcam recognize face

# COLOR BALANCER
def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
    half_percent = percent / 200.0
    channels = cv2.split(img)

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)

        assert len(flat.shape) == 1
        flat = np.sort(flat)
        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img
def contrastInput(img):
    cv2.imshow("Original Image",img)
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Contrast stretching/test1.jpg',img_rescale)
    cv2.imshow("Contrast Image",img_rescale)
    #gray-scale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Gray Image",gray)
    #Applying clahe (Contrast Limited Adaptive Histogram Equalization)
    
    # clahe = cv2.createCLAHE(clipLimit = 5)
    clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    final_img = clahe.apply(gray) + 30
    cv2.imshow("Sharp Clahe",final_img)
    #Enhanced colored Image
    l_channel, a, b = cv2.split(img)
    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((final_img,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2.imshow("Enhanced Image",enhanced_img)
    #Equalization
    equalized = cv2.equalizeHist(gray)
    cv2.imshow("Equalized Image",equalized)
    # Equalization
    img_eq = exposure.equalize_hist(gray)
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Histogram equalization/test2.jpg',img_eq)
    cv2.imshow("Hist Equalized Image",img_eq)
    # Adaptive Equalization
    img_adapteq = exposure.equalize_adapthist(gray, clip_limit=0.03)
    # imwrite('C:/Users/mayan/OneDrive/Desktop/Human-Face-Detection-in-Excessive-Dark-Image-master/CodePython/face/Adaptive equalization/test3.jpg',img_adapteq)
    cv2.imshow("adapteq equalize",img_adapteq)

    #Brightness
    imgBright = increase_brightness(img, value=20)
    cv2.imshow("Bright image",imgBright)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    h, s, v = cv2.split(hsv)
    v += 255
    final_hsv = cv2.merge((h, s, v))
    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow("Colorful Image",image)



    
# loading data.pt file
load_data = torch.load('C:/Users/mayan/OneDrive/Desktop/CyberHack/data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

cam = cv2.VideoCapture("C:/Users/mayan/OneDrive/Desktop/CyberHack/test/test_images/DCPMaam2.jpg")

while True:
    ret, frame = cam.read()
    print(ret)
    # if frame is not None:
      # frame = cv2.resize(frame, (700, 500))

    if not ret:
        print("fail to grab frame, try again")
        break
    # #Enhance Image Pixels
    # sr = cv2.dnn_superres.DnnSuperResImpl_create()
    # path = "/content/gdrive/MyDrive/FaceRecog/facenet-pytorch/resolutionEnhancerModel/EDSR_x4.pb"
    # sr.readModel(path)
    # sr.setModel("edsr",4)
    # result = sr.upsample(frame)
    # # Resized image
    # resized = cv2.resize(result,dsize=None,fx=4,fy=4)
    # cv2.imshow(resized)
    # dst = cv2.detailEnhance(frame, sigma_s=10, sigma_r=0.15)
    # cv2.imshow(dst)
    # contrastInput(frame)
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    print(prob_list)
    boxes, _ = mtcnn.detect(img)
    print(boxes) 
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
        print(prob_list)        
        for i, prob in enumerate(prob_list):
            if prob>0.80:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                if min_dist<0.90:
                    print(box[0],box[1])
                    frame = cv2.putText(frame, name+' '+str(min_dist), (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA, False)
                    print((box[0],box[1]) , (box[2],box[3]))
                frame = cv2.rectangle(frame, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (255,0,0), 2)
                
    cv2.imshow("Recognized Face",frame)
    cv2.waitKey(0)
    # cv2_imshow(frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)
            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()
    

True
[0.99953413 0.99971145]
[[120.55199 137.9341  225.98633 260.84952]
 [403.033   234.04762 504.48837 355.4253 ]]
[0.99953413 0.99971145]
120.55199 137.9341
(120.55199, 137.9341) (225.98633, 260.84952)
403.033 234.04762
(403.033, 234.04762) (504.48837, 355.4253)
False
fail to grab frame, try again
